**TO PERFORM SEMANTIC SEGMENTATION ON THE HOME DEPOT DATASET**



In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import json
import datetime
import sys
import matplotlib.pyplot as plt
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.visualize import display_instances

#The root directory part
root = r"C:\Users\devik\Downloads\HomeDepot\HomeDepotDataset"
sys.path.append(root) 
#The weights file for Mask RCNN which is downloaded
maskrcnn_weights = os.path.join(root, "mask_rcnn_coco.h5")
dir_logs = os.path.join(root, "logs")


# Training of the custom dataset of Home Depot
class CustomConfig(Config):
    name = "object"
    no_images = 2
    # 1 is added in the number of classes as it considers the bakground. 
    num_classes = 1 + 10 
    no_of_epochs = 10
    confidence = 0.85

#Considering the Home Depot dataset to perform Semantic Segmnetation
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        # The 10 categories of Kitchen from the Home Depot website are added to the class.
        self.add_class("object", 1, "Sinks")
        self.add_class("object", 2, "Cabinets")
        self.add_class("object", 3, "Countertops")
        self.add_class("object", 4, "Dinnerware")
        self.add_class("object", 5, "Skillets")
        self.add_class("object", 6, "Cutting Boards")
        self.add_class("object", 7, "Mixing Bowls")
        self.add_class("object", 8, "Toaster Ovens")
        self.add_class("object", 9, "Wall Ovens")
        self.add_class("object", 10, "Garbage Disposals")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Path for the annotations of the dataset which is stored in the json format
        ann = json.load(open(r'C:\Users\devik\Downloads\HomeDepot\HomeDepotDataset\train\annotations.json'))
        data_annotations = list(ann.values()) 

        data_annotations = [a for a in data_annotations if a['regions']]
        for a in annotations:
    
            # Creating a polygon that creates an outline for an instance
            polygons = [r['shape_attributes'] for r in a['regions']] 
            obj = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",obj)
            dict = {"Sinks": 1,"Cabinets": 2,"Countertops": 3,"Dinnerware": 4,"Skillets": 5,"Cutting Boards": 6,"Mixing Bowls": 7,"Toaster Ovens": 8,"Wall Ovens": 9,"Garbage Disposals": 10} #,"xyz": 3}
            num_ids = [dict[a] for a in obj]
            print("numids",num_ids)
            img_path = os.path.join(dataset_dir, a['filename'])
            img = skimage.io.imread(img_path)
            height, width = img.shape[:2]

            self.add_image("object",  
                img_id=a['filename'],  # An unique image id must be used as a filename.
                path=img_path, width=width, height=height, polygons=polygons, num_ids=num_ids)

     """
    def custom_visualize(test_image, model, colors, classes, draw_bbox, mrcnn_visualize, instance_segmentation):
    detections = model.detect([test_image], verbose=1)[0]

    if mrcnn_visualize:
        matplotlib.use('TkAgg')
        visualize.display_instances(test_image, detections['rois'], detections['masks'], detections['class_ids'], classes, detections['scores'])
        return

    if instance_segmentation:
        hsv = [(i / len(detections['rois']), 1, 1.0) for i in range(len(detections['rois']))]
        colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
        random.seed(42)
        random.shuffle(colors)

    for i in range(0, detections["rois"].shape[0]):
        classID = detections["class_ids"][i]

        mask = detections["masks"][:, :, i]
        if instance_segmentation:
            color = colors[i][::-1]
        else:
            color = colors[classID][::-1]

        # To visualize the pixel-wise mask of the object
        test_image = visualize.apply_mask(test_image, mask, color, alpha=0.5)

    test_image = cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR)

    if draw_bbox:
        for i in range(0, len(detections["scores"])):
            (startY, startX, endY, endX) = detections["rois"][i]

            classID = detections["class_ids"][i]
            label = classes[classID]
            score = detections["scores"][i]

            if instance_segmentation:
                color = [int(c) for c in np.array(colors[i]) * 255]

            else:
                color = [int(c) for c in np.array(colors[classID]) * 255]

            cv2.rectangle(test_image, (startX, startY), (endX, endY), color, 2)
            text = "{}: {:.2f}".format(label, score)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(test_image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return test_image
    """


    # The generate_mask() method needs the size of the images to convert polygons to masks.
    def generate_mask(self, img_id):
    
        #The masks generated are a bool array of shape which consists of height, width and instance count with
        #one mask per instance.
        
        img_data = self.img_data[img_id]
        if img_data["source"] != "object":
            return super(self.__class__, self).generate_mask(img_id)

        # Converting the polygons to a bitmap mask of shape.
        data = self.img_data[img_id]
        if data["source"] != "object":
            return super(self.__class__, self).generate_mask(img_id)
        num_ids = data['num_ids']
        mask = np.zeros([data["height"], data["width"], len(data["polygons"])],dtype=np.uint8)
        for i, p in enumerate(data["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	xx, yy = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[xx, yy, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids 

    def img_refer(self, img_id):
        data = self.img_data[img_id]
        if data["source"] == "object":
            return data["path"]
        else:
            super(self.__class__, self).img_refer(img_id)

def train_dataset(model):
    
    # Home Depot model: Training dataset
    train_data = CustomDataset()
    train_data.load_custom(r"C:\Users\devik\Downloads\HomeDepot\HomeDepotDataset", "train")
    train_data.prepare()

    # Home Depot Model: Validation dataset
    val_data = CustomDataset()
    val_data.load_custom(r"C:\Users\devik\Downloads\HomeDepot\HomeDepotDataset", "val")
    val_data.prepare()

    print("The working heads of the training model")
    model.train_dataset(train_data, val_data,learning_rate=config.LEARNING_RATE,epochs=10,layers='heads')
				
							
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,model_dir=dir_logs)
weights = maskrcnn_weights

if not os.path.exists(weights):
  utils.download_trained_weights(weights)

model.load_weights(weights, by_name=True, exclude=[ "mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

#Finally training the model
train(model)	